In [1]:
import h5py
import keras
import numpy as np
import json
import os
import uuid
import yaml

from attlayer import AttentionWeightedAverage
#from avglayer import MaskAverage
from copy import deepcopy
#from finetuning import (sampling_generator, finetuning_callbacks)
from operator import itemgetter
#from global_variables import NB_TOKENS, NB_EMOJI_CLASSES
from keras.layers import *
from keras.layers.merge import concatenate
from keras.layers import Input, Bidirectional, Embedding, Dense, Dropout, SpatialDropout1D, LSTM, Activation
from keras.models import Model, Sequential
from keras.optimizers import Adam
from keras.regularizers import L1L2 
from pathlib import Path
from sklearn.metrics import classification_report, recall_score, precision_score, f1_score
from os.path import exists

Using TensorFlow backend.


In [2]:
def elsa_architecture(nb_classes, nb_tokens, maxlen, feature_output=False, embed_dropout_rate=0, final_dropout_rate=0, embed_dim=300,
                    embed_l2=1E-6, return_attention=False, load_embedding=False, pre_embedding=None, high=False, LSTM_hidden=512, LSTM_drop=0.5):
    """
    Returns the DeepMoji architecture uninitialized and
    without using the pretrained model weights.
    # Arguments:
        nb_classes: Number of classes in the dataset.
        nb_tokens: Number of tokens in the dataset (i.e. vocabulary size).
        maxlen: Maximum length of a token.
        feature_output: If True the model returns the penultimate
                        feature vector rather than Softmax probabilities
                        (defaults to False).
        embed_dropout_rate: Dropout rate for the embedding layer.
        final_dropout_rate: Dropout rate for the final Softmax layer.
        embed_l2: L2 regularization for the embedding layerl.
        high: use or not the highway network
    # Returns:
        Model with the given parameters.
    """
    class NonMasking(Layer):   
        def __init__(self, **kwargs):   
            self.supports_masking = True  
            super(NonMasking, self).__init__(**kwargs)   

        def build(self, input_shape):   
            input_shape = input_shape   

        def compute_mask(self, input, input_mask=None):   
            # do not pass the mask to the next layers   
            return None   

        def call(self, x, mask=None):   
            return x   

        def get_output_shape_for(self, input_shape):   
            return input_shape 
    # define embedding layer that turns word tokens into vectors
    # an activation function is used to bound the values of the embedding
    model_input = Input(shape=(maxlen,), dtype='int32')
    embed_reg = L1L2(l2=embed_l2) if embed_l2 != 0 else None

    if not load_embedding and pre_embedding is None:
        embed = Embedding(input_dim=nb_tokens, output_dim=embed_dim, mask_zero=True,input_length=maxlen,embeddings_regularizer=embed_reg,
                          name='embedding')
    else:
        embed = Embedding(input_dim=nb_tokens, output_dim=embed_dim, mask_zero=True,input_length=maxlen, weights=[pre_embedding],
                          embeddings_regularizer=embed_reg,trainable=True, name='embedding')
    if high:
        x = NonMasking()(embed(model_input))
    else:
        x = embed(model_input)
    x = Activation('tanh')(x)

    # entire embedding channels are dropped out instead of the
    # normal Keras embedding dropout, which drops all channels for entire words
    # many of the datasets contain so few words that losing one or more words can alter the emotions completely
    if embed_dropout_rate != 0:
        embed_drop = SpatialDropout1D(embed_dropout_rate, name='embed_drop')
        x = embed_drop(x)

    # skip-connection from embedding to output eases gradient-flow and allows access to lower-level features
    # ordering of the way the merge is done is important for consistency with the pretrained model
    lstm_0_output = Bidirectional(LSTM(LSTM_hidden, return_sequences=True, dropout=LSTM_drop), name="bi_lstm_0" )(x)
    lstm_1_output = Bidirectional(LSTM(LSTM_hidden, return_sequences=True, dropout=LSTM_drop), name="bi_lstm_1" )(lstm_0_output)
    x = concatenate([lstm_1_output, lstm_0_output, x])
    if high:
        x = TimeDistributed(Highway(activation='tanh', name="high"))(x)
    # if return_attention is True in AttentionWeightedAverage, an additional tensor
    # representing the weight at each timestep is returned
    weights = None
    x = AttentionWeightedAverage(name='attlayer', return_attention=return_attention)(x)
    #x = MaskAverage(name='attlayer', return_attention=return_attention)(x)
    if return_attention:
        x, weights = x

    if not feature_output:
        # output class probabilities
        if final_dropout_rate != 0:
            x = Dropout(final_dropout_rate)(x)

        if nb_classes > 2:
            outputs = [Dense(nb_classes, activation='softmax', name='softmax')(x)]
        else:
            outputs = [Dense(1, activation='sigmoid', name='softmax')(x)]
    else:
        # output penultimate feature vector
        outputs = [x]

    if return_attention:
        # add the attention weights to the outputs if required
        outputs.append(weights)

    return Model(inputs=[model_input], outputs=outputs)

In [3]:
os.environ['CUDA_VISIBLE_DEVICES'] = "1"
cur_lan = "elsa_es"
maxlen = 30
batch_size = 250
lr = 0.001
epoch_size = 25000
nb_epochs = 1000
patience = 5
checkpoint_weight_path = "./ckpt"
loss = "categorical_crossentropy"
optim = "adam"
vocab_path = "/data/elsa"
nb_classes=64

In [4]:
LSTM_hidden = 512
LSTM_drop = 0.5
final_dropout_rate = 0.5
embed_dropout_rate = 0.0
high = False
load_embedding = True
embed_dim = 200

In [5]:
steps = int(epoch_size/batch_size)

wv_path = Path(vocab_path).joinpath("{:s}_wv.npy".format(cur_lan)).as_posix()
X_path = Path(vocab_path).joinpath("{:s}_X.npy".format(cur_lan)).as_posix()
y_path = Path(vocab_path).joinpath("{:s}_y.npy".format(cur_lan)).as_posix()

word_vec = np.load(wv_path, allow_pickle=True)
input_vec, input_label = np.load(X_path, allow_pickle=True), np.load(y_path, allow_pickle=True)
nb_tokens, input_len = len(word_vec), len(input_label)

#please modify the checkpoint_weight_path
#checkpoint_weight_path = '/storage1/user/ss/tmoji_ori/weight/tmoji-lstm-checkpoint-%s-h-1.hdf5' % cur_lan

idx_shuffle = list(range(input_len))
np.random.shuffle(idx_shuffle)
idx_train, idx_val, idx_test = idx_shuffle[ :int(input_len*0.7) ], idx_shuffle[int(input_len*0.7):int(input_len*0.9)], idx_shuffle[int(input_len*0.9):]

(X_train, y_train) = (input_vec[idx_train], input_label[idx_train])
(X_val, y_val) = (input_vec[idx_val], input_label[idx_val])
(X_test, y_test) = (input_vec[idx_test], input_label[idx_test])

In [24]:
for i in range(64):
    print(input_label[:, i].sum())

39295.0
27126.0
26201.0
13569.0
17109.0
10350.0
11106.0
7401.0
6681.0
3546.0
7585.0
6567.0
8581.0
7779.0
8449.0
6747.0
9015.0
5469.0
5369.0
8025.0
5731.0
2105.0
4973.0
8028.0
6007.0
6125.0
4985.0
4671.0
5047.0
5187.0
5017.0
6372.0
5343.0
2831.0
4705.0
3963.0
3633.0
4937.0
5163.0
4205.0
2053.0
2893.0
4043.0
5347.0
4219.0
3767.0
3387.0
4047.0
3445.0
3457.0
3241.0
360.0
3603.0
3509.0
4553.0
1855.0
2923.0
2657.0
3205.0
3529.0
2795.0
3777.0
2189.0
2880.0


In [6]:
model = elsa_architecture(nb_classes=nb_classes, nb_tokens=nb_tokens, maxlen=maxlen, final_dropout_rate=final_dropout_rate, embed_dropout_rate=embed_dropout_rate, 
                          load_embedding=True, pre_embedding=word_vec, high=high, embed_dim=embed_dim)
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 30)           0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 30, 200)      21534400    input_1[0][0]                    
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 30, 200)      0           embedding[0][0]                  
__________________________________________________________________________________________________
bi_lstm_0 (Bidirectio

In [7]:
if optim == 'adam':
    adam = Adam(clipnorm=1, lr=lr)
    model.compile(loss=loss, optimizer=adam, metrics=['accuracy'])
elif optim == 'rmsprop':
    model.compile(loss=loss, optimizer='rmsprop', metrics=['accuracy'])

model.fit(X_train,
          y_train,
          batch_size=batch_size,
          epochs=nb_epochs,
          validation_data=(X_val, y_val),
          callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=patience, verbose=0, mode='auto')],
          verbose=True)

#callbacks = finetuning_callbacks(checkpoint_weight_path, patience, verbose=1)
#for i in range(2):
    #train_gen = sampling_generator(X_train, y_train, batch_size, upsample=False, epoch_size=epoch_size)
    #model.fit_generator(train_gen, steps_per_epoch=steps, epochs=nb_epochs,validation_data=(X_val, y_val),validation_steps=steps, callbacks=callbacks, verbose=True)

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 288912 samples, validate on 82546 samples
Epoch 1/1000
288912/288912 [==============================] - 372s 1ms/step - loss: 2.8115 - acc: 0.3023 - val_loss: 2.3728 - val_acc: 0.4014
Epoch 2/1000
288912/288912 [==============================] - 299s 1ms/step - loss: 2.0686 - acc: 0.4794 - val_loss: 2.0682 - val_acc: 0.4893
Epoch 3/1000
288912/288912 [==============================] - 291s 1ms/step - loss: 1.5835 - acc: 0.5990 - val_loss: 1.9185 - val_acc: 0.5464
Epoch 4/1000
288912/288912 [==============================] - 297s 1ms/step - loss: 1.2863 - acc: 0.6676 - val_loss: 1.8713 - val_acc: 0.5795
Epoch 5/1000
288912/288912 [==============================] - 306s 1ms/step - loss: 1.1113 - acc: 0.7083 - val_loss: 1.8824 - val_acc: 0.6016
Epoch 6/1000
288912/288912 [==============================] - 312s 1ms/step - loss: 1.0035 - acc: 0.7334 - val_lo

In [8]:
_, acc = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=0)

print(acc)

0.6526142373423293


In [9]:
token2index = json.loads(open("/data/elsa/elsa_es_vocab.txt", "r").read())

freq = {line.split()[0]: int(line.split()[1]) for line in open("/data/elsa/elsa_es_emoji.txt").readlines()}
freq_topn = sorted(freq.items(), key=itemgetter(1), reverse=True)[:nb_classes]

In [10]:
y_pred = model.predict(X_test)

In [22]:
print(classification_report(y_test.argmax(axis=1), y_pred.argmax(axis=1), target_names=[e[0] for e in freq_topn]))

              precision    recall  f1-score   support

           😂       0.57      0.72      0.64      3940
           ❤       0.56      0.65      0.60      2746
           😍       0.49      0.65      0.56      2572
           🤣       0.61      0.55      0.58      1321
           😭       0.57      0.65      0.61      1722
           👏       0.73      0.73      0.73      1002
           🔥       0.79      0.72      0.75      1106
           💜       0.75      0.61      0.67       715
           😘       0.51      0.56      0.54       663
           ♀       0.85      0.87      0.86       373
           👇       0.75      0.77      0.76       739
           😈       0.75      0.73      0.74       674
           🔴       0.80      0.81      0.81       869
           🙏       0.75      0.75      0.75       737
           🤦       0.81      0.85      0.83       839
           💪       0.73      0.65      0.69       699
           🤔       0.62      0.66      0.64       958
           💙       0.71    

In [23]:
X_train.shape

(288912, 30)